MIT License

Copyright (c) 2021 Stephanie T. Douglas

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

Cluster membership and parameters used in these plots are taken from Gaia Collaboration et al. (2018b), A&A, 616, A10

https://www.aanda.org/articles/aa/full_html/2018/08/aa32843-18/aa32843-18.html

The original paper is licensed under a CC-BY 4.0 license (https://creativecommons.org/licenses/by/4.0/)

-

Gaia DR2 data used in the plots are taken from Gaia Collaboration et al. (2018a), A&A, 616, A1

https://www.aanda.org/articles/aa/full_html/2018/08/aa33051-18/aa33051-18.html

Accessed through CDS https://vizier.u-strasbg.fr/viz-bin/VizieR-3?-source=I/345/gaia2

Using the X-match service http://cdsxmatch.u-strasbg.fr/

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize,LogNorm
import astropy.units as u
from astropy.table import Table,join
import numpy as np
import astropy.io.ascii as at

Read in the overall cluster data, and fix one of the cluster names so that it matches properly

In [ ]:
cluster_params = at.read("GaiaHRD_tab2.csv")
loc = cluster_params["Cluster"]=="NGC2451A"
cluster_params["Cluster"][loc] = "NGC2451"
# cluster_params

In [ ]:
age_norm = Normalize(vmin=min(cluster_params["log(age)"]),vmax=max(cluster_params["log(age)"]))

# Test plots, nearby clusters

These are nearby clusters where we have to use individual parallaxes for each star

In [ ]:
near_tab = at.read("GaiaHRD_tab1a_DR2xmatch.csv",delimiter=",")

In [ ]:
near_clusters = np.unique(near_tab["Cluster"])

In [ ]:
for cluster in near_clusters:
#     print(cluster)
    plt.figure(figsize=(9,4))
    loc = near_tab["Cluster"]==cluster
    cl_age = cluster_params["log(age)"][cluster_params["Cluster"]==cluster]
#     print(cl_age)
    cl_color = np.ones_like(near_tab["Gmag"][loc])*cl_age[0]
    
    # Plot a CMD with apparent magnitude
    ax = plt.subplot(121)
    ax.scatter(near_tab["bp_rp"][loc],near_tab["phot_g_mean_mag"][loc],marker=".",
               c=cl_color,norm=age_norm,cmap="plasma")
    ax.set_ylim(ax.get_ylim()[::-1])
    ax.set_ylabel("G",fontsize=16)
    ax.set_xlabel(r"G$_{BP}$ - G$_{RP}$",fontsize=16)
    ax.tick_params(labelsize=14)

    # Plot a CMD with absolute magnitude
    ax = plt.subplot(122)
    dist = 1000/near_tab["plx"]
    dmod = 5*np.log10(dist)-5
    ax.scatter(near_tab["bp_rp"][loc],near_tab["phot_g_mean_mag"][loc]-dmod[loc],marker=".",
               c=cl_color,norm=age_norm,cmap="plasma")
    ax.set_ylim(ax.get_ylim()[::-1])
    ax.set_ylabel(r"M$_G$",fontsize=16)
    ax.set_xlabel(r"G$_{BP}$ - G$_{RP}$",fontsize=16)
    ax.tick_params(labelsize=14)

    plt.suptitle(cluster)
    plt.subplots_adjust(wspace=0.35,bottom=0.15)
    
    plt.savefig(f"figures/gaia_HRD_sidebyside_{cluster}.png",
                dpi=600,facecolor="w")

# Test plots, clusters >250 pc away

These clusters are sufficiently far away that we can use one distance for the entire cluster

In [ ]:
far_tab = at.read("GaiaHRD_tab1b_DR2xmatch.csv",delimiter=",")
# near_tab2

In [ ]:
far_clusters = np.unique(far_tab["Cluster"])

In [ ]:
fig = plt.figure(figsize=(9,9))
ax1 = plt.subplot(211)
ax2 = plt.subplot(212)
# age_norm = Normalize(vmin=7.75,vmax=10.2)
for cluster in far_clusters:
#     print(cluster)
    loc = far_tab["Cluster"]==cluster
    cl_age = cluster_params["log(age)"][cluster_params["Cluster"]==cluster]
    if len(cl_age)==0:
#         print("Not found")
        continue
    dmod = cluster_params["DM"][cluster_params["Cluster"]==cluster]
#     print(cl_age)
    cl_color = np.ones_like(far_tab["phot_g_mean_mag"][loc])*cl_age[0]
    
    # Plot a CMD with apparent magnitude
    sc1 = ax1.scatter(far_tab["bp_rp"][loc],far_tab["phot_g_mean_mag"][loc],marker=".",
               c=cl_color,norm=age_norm,cmap="inferno",s=9)
    ax1.set_ylabel("G",fontsize=16)
    ax1.set_xlabel(r"G$_{BP}$ - G$_{RP}$",fontsize=16)

    # Plot a CMD with absolute magnitude
    sc2 = ax2.scatter(far_tab["bp_rp"][loc],far_tab["phot_g_mean_mag"][loc]-dmod,marker=".",
               c=cl_color,norm=age_norm,cmap="inferno",zorder=cl_age,s=9)
    ax2.set_ylabel(r"M$_G$",fontsize=16)
    ax2.set_xlabel(r"G$_{BP}$ - G$_{RP}$",fontsize=16)

ax1.set_ylim(20,5)
ax2.set_ylim(12,-3)
ax1.set_xlim(-0.5,4)
ax2.set_xlim(-0.5,4)
ax1.tick_params(labelsize=14)
ax2.tick_params(labelsize=14)
fig.colorbar(sc2, ax=[ax1,ax2], shrink=0.6,label="log(age/year)")

# Combine lists and animate the plot

The goal here is to create a two-panel plot. The top panel has the CMDs in apparent magnitude, and the bottom panel has the CMDs in absolute magnitude. The clusters are plotted from youngest to oldest, and colored accordingly

In [ ]:
cluster_sort = np.argsort(cluster_params["log(age)"])

In [ ]:
fig = plt.figure(figsize=(9,9))
ax1 = plt.subplot(211)
ax2 = plt.subplot(212)
ax1.set_ylim(21,4)
ax2.set_ylim(14,-5)
ax1.set_xlim(-0.25,4)
ax2.set_xlim(-0.25,4)
ax1.tick_params(labelsize=14)
ax2.tick_params(labelsize=14)

ax1.text(3.95,5,"Figure: Stephanie T. Douglas",horizontalalignment="right")
ax1.text(3.95,6,"Data: Gaia Collaboration et al. (2018)",horizontalalignment="right")

for ct,i in enumerate(cluster_sort):
    cluster = cluster_params["Cluster"][i]
#     print(cluster)
    cl_age = cluster_params["log(age)"][i]
    dmod = cluster_params["DM"][i]
    if dmod<7:
        tab = near_tab
        loc = near_tab["Cluster"]==cluster
        dist = 1000/near_tab["plx"]
        dmod = 5*np.log10(dist[loc])-5
    else:
        tab = far_tab
        loc = far_tab["Cluster"]==cluster
#     print(cl_age)
    cl_color = np.ones_like(tab["phot_g_mean_mag"][loc])*cl_age
    
    # Plot a CMD with apparent magnitude
    sc1 = ax1.scatter(tab["bp_rp"][loc],tab["phot_g_mean_mag"][loc],marker=".",
               c=cl_color,norm=age_norm,cmap="plasma",s=9)
    ax1.set_ylabel("G",fontsize=16)
    ax1.set_xlabel(r"G$_{BP}$ - G$_{RP}$",fontsize=16)

    # Plot a CMD with absolute magnitude
    sc2 = ax2.scatter(tab["bp_rp"][loc],tab["phot_g_mean_mag"][loc]-dmod,marker=".",
               c=cl_color,norm=age_norm,cmap="plasma",zorder=cl_age,s=9)
    ax2.set_ylabel(r"M$_G$",fontsize=16)
    ax2.set_xlabel(r"G$_{BP}$ - G$_{RP}$",fontsize=16)
    
    if ct==0:
        fig.colorbar(sc2, ax=[ax1,ax2], shrink=0.6,label="log(age,yr)")
    
    plt.savefig("figures/gaia_HRD_clusters{0:0>3}.png".format(ct+1),
                bbox_inches="tight",dpi=600,facecolor="w")

Now just plot the absolute magnitude CMDs, still stacked in age order

In [ ]:
fig = plt.figure(figsize=(9,6))
ax2 = plt.subplot(111)
ax2.set_ylim(14,-5)
ax2.set_xlim(-0.25,4)
ax2.tick_params(labelsize=14)
ax2.set_ylabel(r"M$_G$",fontsize=16)
ax2.set_xlabel(r"G$_{BP}$ - G$_{RP}$",fontsize=16)

ax2.text(3.95,-4,"Figure: Stephanie T. Douglas",horizontalalignment="right")
ax2.text(3.95,-3,"Data: Gaia Collaboration et al. (2018)",horizontalalignment="right")

for ct,i in enumerate(cluster_sort):

    cluster = cluster_params["Cluster"][i]
    print(cluster,ct)
    cl_age = cluster_params["log(age)"][i]
    dmod = cluster_params["DM"][i]
    if dmod<7:
        tab = near_tab
        loc = near_tab["Cluster"]==cluster
        dist = 1000/near_tab["plx"]
        dmod = 5*np.log10(dist[loc])-5
    else:
        tab = far_tab
        loc = far_tab["Cluster"]==cluster
#     print(cl_age)
    cl_color = np.ones_like(tab["phot_g_mean_mag"][loc])*cl_age
    
    # Plot a CMD with absolute magnitude
    sc2 = ax2.scatter(tab["bp_rp"][loc],tab["phot_g_mean_mag"][loc]-dmod,marker=".",
               c=cl_color,norm=age_norm,cmap="plasma",zorder=cl_age,s=9)
    if ct==0:
        plt.colorbar(sc2, label="log(age,yr)")
    
    plt.savefig(f"figures/gaia_HRD_clusters_abs_{(ct+1):0>3}_{cluster}.png".format(ct+1),
                bbox_inches="tight",dpi=600,facecolor="w")

# Plot individual clusters 

Plot clusters as individual frames, instead of overlaid on each other. They're still colored by age

In [ ]:
for ct,i in enumerate(cluster_sort):
    fig = plt.figure(figsize=(9,6))
    ax2 = plt.subplot(111)
    ax2.set_ylim(14,-5)
    ax2.set_xlim(-0.25,4)
    ax2.tick_params(labelsize=14)

    ax2.text(3.95,-4,"Figure: Stephanie T. Douglas",horizontalalignment="right")
    ax2.text(3.95,-3,"Data: Gaia Collaboration et al. (2018)",horizontalalignment="right")

    cluster = cluster_params["Cluster"][i]
#     print(cluster)
    cl_age = cluster_params["log(age)"][i]
    dmod = cluster_params["DM"][i]
    if dmod<7:
        tab = near_tab
        loc = near_tab["Cluster"]==cluster
        dist = 1000/near_tab["plx"]
        dmod = 5*np.log10(dist[loc])-5
    else:
        tab = far_tab
        loc = far_tab["Cluster"]==cluster
    if len(np.where(loc)[0])==0:
        print(cluster,"not found?")
    cl_color = np.ones_like(tab["phot_g_mean_mag"][loc])*cl_age
    
    # Pick out the right color for the text
    age_myr = ((10**cl_age)/1e6)
    cmap = cm.get_cmap("plasma")
    age_color = cmap(age_norm(cl_age))
    if cluster=="Trump10":
        cluster=="Trumpler10"
    ax2.text(3.95,-2,f"{cluster}, age = {age_myr:.0f} Myr",color=age_color,horizontalalignment="right")
    
    # Plot a CMD with absolute magnitude
    sc2 = ax2.scatter(tab["bp_rp"][loc],tab["phot_g_mean_mag"][loc]-dmod,marker=".",
               c=cl_color,norm=age_norm,cmap="plasma",zorder=cl_age,s=9)
    ax2.set_ylabel(r"M$_G$",fontsize=16)
    ax2.set_xlabel(r"G$_{BP}$ - G$_{RP}$",fontsize=16)
    
    plt.colorbar(sc2, label="log(age,yr)")
    
    plt.savefig(f"figures/gaia_HRD_{(ct+1):0>3}_{cluster}.png".format(ct+1),
                bbox_inches="tight",dpi=600,facecolor="w")
    plt.close()

Same plot as above, except all the clusters will be plotted in black and not labeled. (Intended for use with students, to have them sort clusters by age)

In [ ]:
for ct,i in enumerate(cluster_sort):
    fig = plt.figure(figsize=(9,6))
    ax2 = plt.subplot(111)
    ax2.set_ylim(14,-5)
    ax2.set_xlim(-0.25,4)
    ax2.tick_params(labelsize=14)

    ax2.text(3.95,-4,"Figure: Stephanie T. Douglas",horizontalalignment="right")
    ax2.text(3.95,-3,"Data: Gaia Collaboration et al. (2018)",horizontalalignment="right")

    cluster = cluster_params["Cluster"][i]
#     print(cluster)
    cl_age = cluster_params["log(age)"][i]
    dmod = cluster_params["DM"][i]
    if dmod<7:
        tab = near_tab
        loc = near_tab["Cluster"]==cluster
        dist = 1000/near_tab["plx"]
        dmod = 5*np.log10(dist[loc])-5
    else:
        tab = far_tab
        loc = far_tab["Cluster"]==cluster
#     print(cl_age)
    cl_color = np.ones_like(tab["phot_g_mean_mag"][loc])*cl_age
    
    # Pick out the right color for the text
    age_myr = ((10**cl_age)/1e6)
    age_color = "k"
    if cluster=="Trump10":
        cluster=="Trumpler10"
#     ax2.text(3.95,-2,f"{cluster}, age = {age_myr:.0f} Myr",color=age_color,horizontalalignment="right")
    
    # Plot a CMD with absolute magnitude
    sc2 = ax2.scatter(tab["bp_rp"][loc],tab["phot_g_mean_mag"][loc]-dmod,marker=".",
               c="k",zorder=cl_age,s=9)
    ax2.set_ylabel(r"M$_G$",fontsize=16)
    ax2.set_xlabel(r"G$_{BP}$ - G$_{RP}$",fontsize=16)
        
    plt.savefig(f"figures/gaia_HRD_black_{(ct+1):0>3}_{cluster}.png".format(ct+1),
                bbox_inches="tight",dpi=600,facecolor="w")
    plt.close()